In [1]:

#from google.colab import drive
#drive.mount('/gd')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import gensim





In [2]:
!pip install gensim
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.8 MB/s eta 0:00:00


In [3]:

df = pd.read_csv("cleaned_data(1).csv")
df.head()
#df.shape



,Unnamed: 0,User,Review,Rating,Likes,Year,virality
0,0,Karen Gulli,Good,3.0,0.0,2024.0,level 1
1,1,Ronny Magadi,"App is useful to certain phone brand ,,,,it is...",1.0,0.0,2024.0,level 1
2,2,Hushikesh Suvakiya,Always promoting anti-Hindu shows.,1.0,8.0,2024.0,level 1
3,3,Mann Official,No I'm not able to remove this app anymore 0 o...,1.0,0.0,2024.0,level 1
4,4,ashutosh gokani,Thy are supporting maharaj movie ...thts not a...,1.0,9.0,2024.0,level 1


In [4]:
df.describe()

,Unnamed: 0,Rating,Likes,Year
count,15892.000000,15891.000000,15891.000000,15891.000000
mean,8140.807576,2.431754,49.261028,2023.075011
std,4698.163198,1.631439,232.236000,1.677576
min,0.000000,1.000000,0.000000,2018.000000
25%,3974.750000,1.000000,0.000000,2023.000000
50%,8235.500000,2.000000,0.000000,2024.000000
75%,12208.250000,4.000000,4.000000,2024.000000
max,16182.000000,5.000000,6990.000000,2024.000000


In [5]:
Review= df.Review.apply(gensim.utils.simple_preprocess)
Review


0                                                   [good]
1        [app, is, useful, to, certain, phone, brand, i...
2                  [always, promoting, anti, hindu, shows]
3        [no, not, able, to, remove, this, app, anymore...
4        [thy, are, supporting, maharaj, movie, thts, n...
                               ...                        
15887    [finally, fixed, my, life, quality, has, impro...
15888    [too, many, bugs, for, paid, service, releases...
15889    [wont, load, it, just, says, netflix, and, the...
15890    [exceptional, great, selection, streaming, and...
15891    [love, my, netflix, just, got, it, few, months...
Name: Review, Length: 15892, dtype: object

In [6]:
model = gensim.models.Word2Vec(
    window=8,
    min_count=2,
    workers=5,
)

In [7]:
model.build_vocab(Review, progress_per= 800)

In [8]:
model.train(Review, total_examples=model.corpus_count, epochs=model.epochs)

(2254266, 3095940)

In [9]:
model.save("./word2vec-documentory-review.model")

In [10]:
model.wv.most_similar("bad")

[('disappointing', 0.8416649103164673),
 ('poor', 0.8155046701431274),
 ('disappointed', 0.8092042207717896),
 ('useful', 0.7952793836593628),
 ('cool', 0.7867085933685303),
 ('irritating', 0.777596116065979),
 ('thankful', 0.7728807926177979),
 ('dissatisfied', 0.7516167759895325),
 ('brilliant', 0.7510230541229248),
 ('subtleties', 0.7424894571304321)]

In [11]:
model.wv.similarity(w1="cheap", w2="boring")

0.7691173

In [12]:
def rev_embed(x):  # Pass the model as an argument
    x = x.split(' ')
    embedd = []
    for i in x:
        if i in model.wv:  # Check if word is in vocabulary
            embedd.append(model.wv[i]) # size  of model.wv[i] is 100
    if embedd:  # If any embeddings were found
        x = np.array(embedd)
        x = np.mean(x, axis=0)
        return x
    else:
        return np.zeros(model.vector_size)  # Return a zero vector for missing words


In [13]:
feat_vect = []
for i in df['Review']:
  feat_vect.append(rev_embed(i))

In [15]:
vect_hash = {}
for i in range(len(df)):
  name = str(df["Rating"][i])+ " " + str(df["virality"][i])
  vect_hash[name] = rev_embed(df["Review"][i]).tolist()

vect_hash


{'3.0 level 1': [-0.2778792679309845,
  0.32405713200569153,
  -0.016967736184597015,
  0.022372273728251457,
  0.10876602679491043,
  0.03489464893937111,
  0.11714878678321838,
  0.6921056509017944,
  -0.48631030321121216,
  -0.028487853705883026,
  0.20186614990234375,
  -0.4193727374076843,
  0.08630286157131195,
  0.35698747634887695,
  0.03608500584959984,
  -0.25252845883369446,
  0.09522176533937454,
  -0.1501653641462326,
  -0.25925835967063904,
  -0.44547852873802185,
  0.2986016869544983,
  -0.0710262581706047,
  0.12583084404468536,
  -0.33995115756988525,
  0.06888297200202942,
  0.045866355299949646,
  -0.457229346036911,
  -0.17090584337711334,
  -0.34193697571754456,
  -0.26223716139793396,
  0.010637548752129078,
  0.13873440027236938,
  0.2513139545917511,
  -0.4991109073162079,
  -0.3648657202720642,
  0.4434507489204407,
  0.10290279984474182,
  -0.3084487020969391,
  -0.3781460225582123,
  0.005112520419061184,
  0.20019103586673737,
  -0.4287654459476471,
  0.1546

In [16]:
feat_vect=np.array(feat_vect)
feat_vect

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.39164209,  0.43400809, -0.12502518, ...,  0.0718051 ,
        -0.22836129, -0.28419122],
       [-0.08839153,  0.02729439,  0.07939436, ..., -0.14395641,
         0.02596104,  0.03485791],
       ...,
       [-0.45090464,  0.52665854, -0.38307625, ..., -0.13879405,
         0.36662355, -0.72086555],
       [-0.28393838,  0.44756487,  0.00603553, ..., -0.12133714,
         0.09308331, -0.02328027],
       [ 0.34912342, -0.12752135,  0.52580082, ..., -1.03648198,
        -0.35500059,  0.12091168]])

In [17]:
feat_vect.shape

(15892, 100)

In [18]:
np.save('./feat_vect_word2vec.npy', feat_vect)